# Algorithms for Material Flow Matrices

In [80]:
import numpy
import math
from pprint import pprint

# Overview of the process flow through individual departments 

class MFM:
    def __init__(self, product_groups: list[int], number_of_transportation: list[int], process_flow: list[int], custom_order: list[str] = None):

        """
        Initialize MFM class with product groups, number of transportation, and process flow.
        """
        print("Material Flow Matrix created successfully")
        self.product_groups = product_groups
        self.number_of_transportation = number_of_transportation
        self.process_flow = process_flow
        self.custom_order = custom_order
    
    @staticmethod
    def pprint_list(list_to_print: list, column_width: int = 10, visual_string: str = None) -> int:
        row_string = ""
        ltp = list_to_print
        for e in range(len(ltp)):
            if visual_string and e != len(ltp)-1:
                row_string += str(f"{ltp[e]}{visual_string}{' '*(column_width-len(str(ltp[e]))-len(str(visual_string)))}")
            else:
                row_string += str(f"{ltp[e]}{' '*(column_width-len(str(ltp[e])))}")
        print(row_string)
        return len(row_string)
    
    @staticmethod
    def pprint_r_chart(r_chart: list, column_width: int = 10) -> None:
        l = MFM.pprint_list(r_chart[0], column_width)
        for e in range(len(r_chart)):
            MFM.pprint_list(r_chart[e], column_width) if e!=0 else None
            print("-"*l) if e == 0 else None
        print("-"*l)

    
    @staticmethod
    def get_nodes(product_g: list[str], process_flow: list[str]):
        nodes = {}
        ipf = [flow.split() for flow in process_flow] 
        for pg in range(len(product_g)):
            temp_list_of_nodes = []
            for i_ in range(len(ipf[pg])-1):
                temp_list_of_nodes.append((ipf[pg][i_], ipf[pg][i_+1]))
            nodes[product_g[pg]] = temp_list_of_nodes

        return nodes
    
    def _transform_flows(flows: list[str], custom_o: list[str] = None) -> list[list[str]]:
        """
        flows: all available workflows of the products

        returns:
            representative_flow
            max_flow_size
            flow_list
        """
        # TODO what if there is no flow that holds ALL stations?

        ipf = [flow.split() for flow in flows] 

        sizes = [len(i) for i in ipf]
        size = max(sizes)

        rep_flow = custom_o if custom_o else sorted(ipf[sizes.index(size)])

        return rep_flow, size, ipf

    
    def make_relationship_chart(product_g: list[str], number_of_t: list[int], process_f: list[str], custom_o: list[str] = None) -> numpy.array:
        
        """
        Generates a relationship chart based on the provided data.

        Parameters:
        - product_g (list[str]): A list of strings representing products.
        - number_of_t (list[int]): A list of integers representing the number of transactions for each product.
        - process_f (list[str]): A list of strings representing process flows.
        - custom_o (list[str], optional): A list of strings representing custom flows. Defaults to None.

        Returns:
        - numpy.array: A numpy array representing the relationship chart.
        - list: A list of tuples representing undirected nodes.
        - list: A list of integers representing the weights of the unweighted nodes.

        The function generates a relationship chart based on the provided products, number of transactions,
        process flows, and custom flows. It calculates the relationship between nodes and assigns a category
        based on the sum of transactions. The category is determined by the following ranges:
        - 'A' (Absolutely necessary): 45 to 70
        - 'E' (Essential necessary): 25 to 44
        - 'I' (Important): 15 to 24
        - 'O' (Out of importance): 5 to 14
        - 'U' (Unimportant): 0 to 4

        If custom flows are provided, they will be used in the calculation. The function returns the generated
        relationship chart as a numpy array, a list of undirected nodes, and a list of weights for unweighted nodes.
        """
        
        rep_flow, size, ipf = MFM._transform_flows(process_f, custom_o)

        ## == new method
        matrix = []
        # print(rep_flow)

        # Create the base chart YIKES
        for row in range(size+1):
            # print("row ", row)
            matrix.append([]) if row != size else None
            for col in range(size-row):
                # print("col ", col)
                if row==0:
                    if col==0:
                        matrix[row].append("") # why...
                    else:
                        matrix[row].append(rep_flow[size-col])
                else:
                    if col == 0:
                        matrix[row].append(rep_flow[row-1])
                        matrix[row].append(0) # this is complete bullcrap and it will fail at one point
                    else:
                        matrix[row].append(0)

        # Fill up chart with connection sums
        print(size)

        all_nodes = MFM.get_nodes(product_g,process_f)

        def get_category(score):

            # Define categories
            categories = {
                'A': (45, 70), # Absolut notwendig
                'E': (25, 44), # Erforderlich
                'I': (15, 24), # Immer noch wichtig
                'O': (5, 14), # Ohne große Bedeutung
                'U': (0, 4) # Unwichtig
            }
            
            for category, (lower, upper) in categories.items():
                if lower <= score <= upper:
                    return category
            return None
        
        undirected_nodes_list = []
        unweighted_nodes_weights = []

        for row in range(1, size+1):
            for col in range(1, size+1-row):
                if col <= (size-row+1):
                    # Check the nodes and sum up their value
                    l1, l2 = rep_flow[row-1], rep_flow[size-col]
                    nodes_to_check = [(l1,l2), (l2,l1)]

                    temp_sum = 0
                    for n in nodes_to_check:
                        for p in range(len(product_g)):
                            # TODO what if a node is there multiple times ? is that even possible?
                            temp_sum += number_of_t[p] if n in all_nodes[product_g[p]] else 0

                    node_category = get_category(temp_sum) 
                    matrix[row][col] = node_category #if node_category in ['A','E','I'] else ""
                    # matrix[row][col] = temp_sum if temp_sum != 0 else ""

                    undirected_nodes_list.append(nodes_to_check[0]) if temp_sum != 0 else None
                    unweighted_nodes_weights.append(temp_sum) if temp_sum != 0 else None
       
        return matrix, undirected_nodes_list, unweighted_nodes_weights

    def make_matrix(product_g: list[str], number_of_t: list[int], process_f: list[str], custom_o: list[str] = None) -> numpy.array:
        """
        Create a matrix representing the flow of transportation between product groups.
        
        Parameters:
            product_g (list[str]): List of product groups.
            number_of_t (list[int]): List of number of transportation for each product group.
            process_f (list[str]): List of process flows between product groups.
        
        Returns:
            numpy.array: Matrix representing the flow of transportation between product groups.
        """
        # ipf = individual process flows

        rep_flow, size, ipf = MFM._transform_flows(process_f, custom_o)

        array_size = (size+2 ,size+2)
        matrix = numpy.full(array_size, None)

        # Create the nodes for each product group
        nodes = MFM.get_nodes(product_g, process_f)

        # Fill up the cells of the matrix
        # Sums are made in a running fashion, since the filler works up->down and left->right

        for row in range(len(matrix[0])):
            if row == 0:
                for col in range(len(matrix)):
                    matrix[row, col] = "from/ to" if col == 0 else "Σ from" if col == size+1 else rep_flow[col-1]

            elif row == size+1:
                for col in range(len(matrix)):        
                    if col == 0:
                            matrix[row, col] = "Σ to"
                    elif col == size+1:
                        matrix[row+1, col] = math.inf
                    else:
                        # Sum up the column
                        column_sum = 0
                        for row in range(1, len(matrix[0])-1):
                            # print(matrix[row][col])
                            column_sum += matrix[row][col] if type(matrix[row][col])==type(1) else 0
                        matrix[row+1, col] = column_sum
            
            else: 
                for col in range(len(matrix)):
                    if col == row:
                        matrix[row,col] = math.inf
                    
                    elif col == 0:
                        # print(row,col)
                        matrix[row,col] = rep_flow[row-1]

                    # sum up the row
                    elif col == size+1:
                        row_sum = sum([value for value in matrix[row] if type(value)==type(1)])
                        matrix[row,col] = row_sum
        
                    else:
                        # filling in the values
                        temp_field_sum = 0
                        for p in range(len(product_g)):
                            # TODO what if a node is there multiple times ?
                            temp_field_sum += number_of_t[p] if (matrix[row][0],matrix[0][col]) in nodes[product_g[p]] else 0
                        matrix[row,col] = temp_field_sum if temp_field_sum != 0 else "-"

        return matrix
    
    @staticmethod
    def pprint_matrix(mf_matrix, column_width = 10) -> int:
        """
        Pretty print the matrix.
        
        Parameters:
            mf_matrix (numpy.array): Matrix to be pretty printed.
            column_width (int): width of columns = default 10
        """
        m_size = len(mf_matrix)
        l = column_width * m_size
        for i in range(m_size):
            if i==0 or i == 1 or i == m_size-1:
                print("-"*l)
            l = MFM.pprint_list(mf_matrix[i], column_width=column_width)
        print("-"*l)

    @staticmethod
    def sums(mfm) -> tuple[int,int]:
        # what about using the fucking EXISTING row sum mechanism?

        sum_above_diagonal, sum_below_diagonal = 0, 0
        
        # Sum of all values above the diagonal line: 
        for col in range(2, len(mfm)-1):
            for row in range(1,len(mfm)-2):
                sum_above_diagonal += mfm[row][col] if (col > row) and type(mfm[row][col])==type(1) else 0
            
        # Sum of all values below the diagonal line
        for col in range(1, len(mfm)-1):
            for row in range(1,len(mfm)-1):
                sum_below_diagonal += mfm[row][col] if (col < row) and type(mfm[row][col])==type(1) else 0
        
        # 
        return (sum_above_diagonal, sum_below_diagonal)

    @staticmethod
    def order_algorithm(mf_matrix) -> list:

        """
        The aim here is to find the best material flow (or-
        der of departments) for manufacturing the product 
        groups.

        Procedure: 
        - Calculate the row and column totals. 
        - Calculate the quotients of the row and column totals for each matrix element: 

        ( ∑ column values / ∑ row values ) = Q_ZS

        - Find the largest quotient Q_ZS_max. 
        - Eliminate the row and column with the largest quotient from matrix. 
        - Add the eliminated matrix elements to an order 
        list. 

        Now repeat these steps with the remaining matrix 
        elements.
        """

        master_order = []

        m_size = len(mf_matrix)

        for iteration in range(len(mf_matrix)-2):

            # Update matrix size
            m_size = len(mf_matrix)

            # Calculate the row and column totals. 
            column_sums = []
            row_sums = []

            for row in range(1, m_size-1):
                row_sum = sum([mf_matrix[row][i] for i in range(1,m_size-1) if type(mf_matrix[row][i])==type(1)])

                # Update matrix, sums for calculation
                mf_matrix[row,m_size-1] = row_sum
                row_sums.append(row_sum)
                
            for col in range(1, m_size-2):
                column_sum = 0
                for row in range(1, len(mf_matrix[0])-1):
                    column_sum += mf_matrix[row][col] if type(mf_matrix[row][col])==type(1) else 0

                # Update matrix
                mf_matrix[m_size-1, col] = column_sum
                column_sums.append(column_sum)

            # Calculate the quotients of the row and column totals for each matrix element
            def Q_ZS(i,mxm_matrix, limit):
                s = len(mxm_matrix)
                e = 0.0000000000001
                result = round(mf_matrix[i][s-1]/(mf_matrix[s-1][i]+e),1)
                return result if result < limit else math.inf

            lfq = (sum(column_sums)+sum(row_sums))*10
            quotients = [Q_ZS(i, mf_matrix, lfq) if i>0 else "Quotient" for i in range(m_size-1)]

            # Find the largest quotient Q_ZS_max
            Q_ZS_max = max(quotients[1:])
            index_QZS_max = quotients[1:].index(Q_ZS_max)

            # Display current state of the matrix
            if iteration >0:
                print(f"\n New Matrix after iteration {iteration}")
            else:
                pass

            MFM.pprint_matrix(mf_matrix)
            quotients_tp = quotients
            quotients_tp[index_QZS_max+1] = f"|{quotients_tp[index_QZS_max+1]}|"
            MFM.pprint_list(quotients_tp)

            # Add the eliminated matrix elements to an order list
            master_order.append(mf_matrix[0][index_QZS_max+1])

            # Eliminate the row and column with the largest quotient from matrix
            mf_matrix = numpy.delete(mf_matrix, index_QZS_max+1, axis=1)
            mf_matrix = numpy.delete(mf_matrix, index_QZS_max+1, axis=0)

            # print("\n Order = ", master_order,"\n")
            
        return master_order
    
product_group = ["I","II","III","IV","V","VI","VII"]
number_of_transportation = [20,20,25,20,5,5,5]
process_flow = ["A B C D E F G H I","A C D F G I","A D B E H F I","A C D B E G I","A E F G H I","A D C B F G H I","A C D H D G I"]
# process_flow = ["A B C D E F G I H","I G B I B","A D B A H F I","A C I A E G I","A H A G H I","A D C B F G H I","G C D A I"]
material_flow_matrix = MFM.make_matrix(product_group,number_of_transportation,process_flow)
# length_fl = MFM.pprint_matrix(material_flow_matrix)


best_order = MFM.order_algorithm(material_flow_matrix)

print("\n Final best order")
MFM.pprint_list(best_order, column_width= 5, visual_string= " -> ")


print()
# Display resulting, optimized matrix

optimized_matrix = MFM.make_matrix(product_group, number_of_transportation, process_flow, custom_o=best_order)
print("DEFAULT MATRIX")
MFM.pprint_matrix(material_flow_matrix)
print("OPTIMIZED MATRIX")
MFM.pprint_matrix(optimized_matrix)

--------------------------------------------------------------------------------------------------------------
from/ to  A         B         C         D         E         F         G         H         I         Σ from    
--------------------------------------------------------------------------------------------------------------
A         inf       20        45        30        5         -         -         -         -         100       
B         -         inf       20        -         45        5         -         -         -         70        
C         -         5         inf       65        -         -         -         -         -         70        
D         -         45        5         inf       20        20        5         5         -         100       
E         -         -         -         -         inf       25        20        25        -         70        
F         -         -         -         -         -         inf       50        -         25        75        
G

In [81]:
all_nodes = MFM.get_nodes(product_group,process_flow)
pprint(all_nodes)

{'I': [('A', 'B'),
       ('B', 'C'),
       ('C', 'D'),
       ('D', 'E'),
       ('E', 'F'),
       ('F', 'G'),
       ('G', 'H'),
       ('H', 'I')],
 'II': [('A', 'C'), ('C', 'D'), ('D', 'F'), ('F', 'G'), ('G', 'I')],
 'III': [('A', 'D'),
         ('D', 'B'),
         ('B', 'E'),
         ('E', 'H'),
         ('H', 'F'),
         ('F', 'I')],
 'IV': [('A', 'C'), ('C', 'D'), ('D', 'B'), ('B', 'E'), ('E', 'G'), ('G', 'I')],
 'V': [('A', 'E'), ('E', 'F'), ('F', 'G'), ('G', 'H'), ('H', 'I')],
 'VI': [('A', 'D'),
        ('D', 'C'),
        ('C', 'B'),
        ('B', 'F'),
        ('F', 'G'),
        ('G', 'H'),
        ('H', 'I')],
 'VII': [('A', 'C'),
         ('C', 'D'),
         ('D', 'H'),
         ('H', 'D'),
         ('D', 'G'),
         ('G', 'I')]}


In [82]:
print(MFM.sums(material_flow_matrix)) # Correct (390, 200)
print(MFM.sums(optimized_matrix)) # Correct (415, 175)

(535, 85)
(565, 55)


## 2.3 Relationship chart

In [83]:
relationship_chart, ud_nodes, ud_nodes_weights = MFM.make_relationship_chart(product_group,number_of_transportation,process_flow)
# print(relationship_chart)
MFM.pprint_r_chart(relationship_chart)

9
          I         H         G         F         E         D         C         B         
------------------------------------------------------------------------------------------
A         U         U         U         U         O         E         A         I         
B         U         U         U         O         A         A         E         
C         U         U         U         U         U         A         
D         U         O         O         I         I         
E         U         E         I         E         
F         E         E         A         
G         A         E         
H         E         
------------------------------------------------------------------------------------------
